In [1]:
import os 
import time 
import random
import numpy as np 

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard

from data_prepare.save_data import save
from data_prepare.data_splitter import get_train_and_test_data as gttd
from service.service_data import ServiceData as sd

In [2]:
level = 'SkiFall'
save(level, delete=False, debug=True)
os.chdir('D:/Documents/Python/fallguy_bot/DATA/LEVELS/'+ level)

Already saved


In [7]:
X = np.load(open('X.npy','rb'))
y = np.load(open('y.npy','rb'))

print(X.shape)
print(y.shape)


(9704, 120, 200, 1)
(9704, 7)


In [4]:
def only_w(array):
    new_arr = array[1:]
    if any(new_arr):
        return False
    if array[0] == 1:
        return True

In [5]:
nothing_counter = 0
w_counter = 0
for lable in y:
    if not any(lable):
        nothing_counter += 1
    if only_w(lable):
        w_counter += 1
        
print(nothing_counter)
print(w_counter)
        

252
4607


In [6]:
X_train, y_train, X_test, y_test = gttd(X,y)

press_number in key = {'W_nums': 0, 'A_nums': 1757, 'S_nums': 127, 'D_nums': 1977, 'SPACE_nums': 221, 'E_nums': 218, 'nothing': 4859}
max press = 4859
In Key = A_nums, press len list = 4859
In Key = S_nums, press len list = 4859
In Key = D_nums, press len list = 4859
In Key = SPACE_nums, press len list = 4859
In Key = E_nums, press len list = 4859
In Key = nothing, press len list = 4859
img_number = 29153
creating X_test
creating y test
creating X_train
creating y_train
try to return


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
del X, y
X_train = X_train/255.0
X_test = X_test/255.0

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')




In [ ]:
what_label = y_train
new_dict_key = dict(list(sd.keys_categories.items())[1:-1])
only_nothing = 0
for key_pos, key_name in enumerate(new_dict_key):
    press_number = 0 
    for label in what_label:
        for index, num in enumerate(label):
            if num == 1 and index==key_pos:
                press_number += 1
    print(f'In {len(what_label)} categories. You pressed key with position {key_name}:')
    print(f'{press_number} times')
    print('its have {:.2f} percent'.format(press_number*100/len(what_label)))
    print(f'________________________________________________________________________')
for lable in what_label:
    if not any(lable):
        only_nothing += 1  
    
    
print(f'In {len(what_label)} categories. You pressed do nothing')
print(f'{only_nothing} times')
print('its have {:.2f} percent'.format(only_nothing*100/len(what_label)))
print(f'________________________________________________________________________')


In [ ]:
X_train.shape[1:]
y_train.shape[1:]


In [ ]:
#My fucked up model 
# model = Sequential()

# model.add(Conv2D((64), (3,3), input_shape = X_train.shape[1:]))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D((32), (2,2)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())
# model.add(Dense(8))
# model.add(Activation('relu'))


# model.add(Dense(7))
# model.add(Activation('sigmoid'))

# model.compile(
#     loss="binary_crossentropy",
#     optimizer='adam',
#     metrics=['accuracy'],
# )
# model.summary()

model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5), padding="same",
	input_shape=X_train.shape[1:], kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same",
	kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(500, kernel_regularizer=l2(0.)))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(len(y_train[0]), kernel_regularizer=l2(0.)))
model.add(Activation("softmax"))

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
NAME = 'CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard])
model.save(f'cnn_{level}.model')